In [1]:
%matplotlib inline
import pandas as pd
import zipfile
import shutil
import urllib2
from urllib2 import urlopen
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as dates
import matplotlib.ticker as tick
import scipy.stats as sp
import statsmodels.api as sm
from pandas.stats.api import ols
from datetime import datetime
from bs4 import BeautifulSoup
from pylab import rcParams
import platform
rcParams['figure.figsize'] = 15, 10
import re
import os
import sys
import glob
import urllib
import HTMLParser
from cStringIO import StringIO

import pyproj
from pyproj import Proj

import gzip

import ftplib
import calendar
import datetime
from datetime import date

import pymodis

WxPython missing, no GUI enabled


In [2]:
'''
Locate ArcPy and add it to the path
Created on 13 Feb 2015
@author: Jamesramm
https://github.com/JamesRamm/archook/blob/master/archook.py
'''
import _winreg
import sys
from os import path
def locate_arcgis():
  '''
  Find the path to the ArcGIS Desktop installation.
  Keys to check:
  HLKM/SOFTWARE/ESRI/ArcGIS 'RealVersion' - will give the version, then we can use
  that to go to
  HKLM/SOFTWARE/ESRI/DesktopXX.X 'InstallDir'. Where XX.X is the version
  We may need to check HKLM/SOFTWARE/Wow6432Node/ESRI instead
  '''
  try:
    key = _winreg.OpenKey(_winreg.HKEY_LOCAL_MACHINE,
                          'SOFTWARE\\Wow6432Node\\ESRI\\ArcGIS', 0)

    version = _winreg.QueryValueEx(key, "RealVersion")[0][:4]

    key_string = "SOFTWARE\\Wow6432Node\\ESRI\\Desktop{0}".format(version)
    desktop_key = _winreg.OpenKey(_winreg.HKEY_LOCAL_MACHINE,
                                  key_string, 0)

    install_dir = _winreg.QueryValueEx(desktop_key, "InstallDir")[0]
    return install_dir
  except WindowsError:
    raise ImportError("Could not locate the ArcGIS directory on this machine")

def get_arcpy():  
  '''
  Allows arcpy to imported on 'unmanaged' python installations (i.e. python installations
  arcgis is not aware of).
  Gets the location of arcpy and related libs and adds it to sys.path
  '''
  install_dir = locate_arcgis()  
  arcpy = path.join(install_dir, "arcpy")
  # Check we have the arcpy directory.
  if not path.exists(arcpy):
    raise ImportError("Could not find arcpy directory in {0}".format(install_dir))

  # First check if we have a bin64 directory - this exists when arcgis is 64bit
  bin_dir = path.join(install_dir, "bin64")
  if not path.exists(bin_dir):
    # Fall back to regular 'bin' dir otherwise.
    bin_dir = path.join(install_dir, "bin")

  scripts = path.join(install_dir, "ArcToolbox", "Scripts")  
  sys.path.extend([arcpy, bin_dir, scripts])

get_arcpy()
import arcpy
arcpy.CheckOutExtension("spatial")
from arcpy import env 
from arcpy.sa import *

In [3]:
print("Operating System " + platform.system() + " " + platform.release())
print("Python Version " + str(sys.version))
print("Pandas Version " + str(pd.__version__))
print("Numpy Version " + str(np.__version__))

Operating System Windows 7
Python Version 2.7.10 (default, May 23 2015, 09:40:32) [MSC v.1500 32 bit (Intel)]
Pandas Version 0.19.1
Numpy Version 1.11.2


In [4]:
def replace_hdr_file(hdrfile):
    """
    Replace the .hdr file for a .bil raster with the correct data for Arc processing
    Required: hdrfile -- filepath for .hdr file to replace/create
    Output:   None
    """
    # hdr file replacment string
    HDRFILE_STRING = "byteorder M\n\
    layout bil\n\
    nbands 1\n\
    nbits 16\n\
    ncols 6935\n\
    nrows 3351\n\
    ulxmap -124.729583333331703\n\
    ulymap 52.871249516804028\n\
    xdim 0.00833333333\n\
    ydim 0.00833333333\n"
    with open(hdrfile, 'w') as o:
        o.write(HDRFILE_STRING)

In [7]:
path= "H:/GIS/MODIS2/MODIS.gdb/"
arcpy.env.workspace = path

for rast in arcpy.ListRasters()[:3]:
        dsc = arcpy.Describe(rast)
        print(dsc.baseName) 

PET200006c
PET200007c
PET200008c


In [ ]:
for hdrfile in glob.glob("H:/GIS/SNODAS/SNODASUNZIPPED/SNODASUNGZ/*.Hdr"):
    replace_hdr_file(hdrfile)

In [ ]:
sr = arcpy.SpatialReference(4326) #Spatial Projection WGS84

# Set output coordinate system
outCS = arcpy.SpatialReference('NAD 1983 UTM Zone 12N')  #NAD83 Zone 12 Code is 26912

# Set local variables
inMaskData = "H:/GIS/SNODAS/SNODAS.gdb/UT_HUC_area"

# from Table 4 of http://nsidc.org/data/docs/noaa/g02158_snodas_snow_cover_model/
prodcodelong = {1025: 'Precipitation', 1034: 'Snow water equivalent', 1036: 'Snow depth', 
            1038: 'Snow pack average temperature', 1039: 'Blowing snow sublimation',
            1044: 'Snow melt', 1050: 'Snow pack sublimation'}
prodcode = {1025: 'PREC', 1034: 'SWEQ', 1036: 'SNOD', 1038: 'SPAT', 1039: 'BSSB', 1044: 'SNML', 1050: 'SPSB'}

# iterate through data files creating rasters and defining projections
for dtfile in glob.glob("H:/GIS/SNODAS/SNODASUNZIPPED/SNODASUNGZ/*.dat"):
    indata = arcpy.Raster(dtfile)
    arcpy.DefineProjection_management(indata, sr)
    typAbb = prodcode[int(os.path.basename(dtfile)[8:12])]
    
    #indata.save("H:/GIS/SNODAS/SNODAS.gdb/" + typAbb + dtfile[-20:-11]) #YYYYMMDD
    
    # Execute ExtractByMask to clip snodas data to Utah watersheds
    outExtractByMask = arcpy.sa.ExtractByMask(indata, inMaskData)

    # Determine the new output feature class path and name = productcode + YYYYMMDD and save to a geodatabase
    outrast = "H:/GIS/SNODAS/SNODAS.gdb/" + typAbb + dtfile[-20:-11] #os.path.join(outWorkspace, rast)
    
    # Print name to veryfy save
    print(typAbb + dtfile[-20:-11])
    
    # Project Raster to UTM Zone 12
    arcpy.ProjectRaster_management(outExtractByMask, outrast, outCS, 'BILINEAR', '1000',\
                                   'WGS_1984_(ITRF00)_To_NAD_1983', '#', '#')
    

From http://nsidc.org/data/docs/noaa/g02158_snodas_snow_cover_model/, the file abbreviations are as follows:
<ul type="disc">
<li>`RAIN` = `Wet Precip`</li>
<li>`SWEQ` = `Snow Water Equivalent`</li> 
<li>`SNOD` = `Snow Depth`</li> 
<li>`SPAT` = `Snow Pack Average Temp`</li>
<li>`BSSB` = `Blowing Snow Sublimation`</li>
<li>`SNML` = `Snowmelt`</li>
<li>`SPSB` = `Snow Pack Sublimation`</li>
</ul>

In [27]:
prodcode = {'us_ssmv11038wS__A':'SPAT', 'us_ssmv11044bS__T':'SNML', 'us_ssmv11050lL00T':'SPSB', 
            'us_ssmv11034tS__T':'SWEQ', 'us_ssmv01025SlL00':'RAIN', 'us_ssmv01025SlL01':'SNOW',
            'us_ssmv11036tS__T':'SNOD', 'us_ssmv11039lL00T':'BSSB'}

path = "H:/GIS/SNODAS/geotifSNODAS/SNDS/"
for filename in os.listdir(path):
    if filename.startswith("us_ssmv"):
        code = prodcode[filename[0:17]]
        yrsrt = filename.find('TNATS') + 5
        yr = filename[yrsrt:yrsrt+4]
        mo = filename[yrsrt+4:yrsrt+6]
        dy = filename[yrsrt+6:yrsrt+8]
        os.rename(os.path.join(path, filename), os.path.join(path,code+yr+mo+dy+filename[-4:]))

In [24]:
def mergeRasts(path, data_type = 'AET', monthRange = [1,4], yearRange = [2000,2001]):
    arcpy.env.workspace = path
    print(arcpy.ListRasters())
    
    for y in range(yearRange[0],yearRange[-1]+1): #set years converted here
        for m in range(monthRange[0],monthRange[-1]+1): #set months converted here   
            nm = data_type + str(y) + str(m).zfill(2)
            rlist=[]
            for rast in arcpy.ListRasters(nm+'*'): 
                rlist.append(rast)
            print(rlist)

In [25]:
path="H:/GIS/MODIS2/MODIS.gdb"
mergeRasts(path)

[]
[u'AET2000018', u'AET2000019']
[u'AET2000028', u'AET2000029']
[u'AET2000038', u'AET2000039']
[u'AET2000048', u'AET2000049']
[u'AET2001018', u'AET2001019']
[u'AET2001028', u'AET2001029']
[u'AET2001038', u'AET2001039']
[u'AET2001048', u'AET2001049']


In [32]:
path = "H:/GIS/SNODAS/geotifSNODAS/SNDS/"
g = {}
from arcpy.sa import *
arcpy.env.workspace = path
arcpy.env.overwriteOutput = True

code = 'PREC'

for y in range(2003,2016):
    for m in range(1,13):
        g[code+str(y)+str(m).zfill(2)] = []
        for name in sorted(glob.glob(path+code+'*.tif')):
            rast = os.path.basename(name)
            if rast[0:4] == code and int(rast[4:8]) == y and int(rast[8:10]) == m:
                g[code+str(y)+str(m).zfill(2)].append(rast)
            else:
                pass
        if len(g[code+str(y)+str(m).zfill(2)])>0:
            print(g[code+str(y)+str(m).zfill(2)])
            calc = CellStatistics(g[code+str(y)+str(m).zfill(2)], statistics_type = "SUM", ignore_nodata="DATA")
            calc.save("H:/GIS/SNODAS/SNODAS.gdb/"+rast[0:4]+str(y).zfill(2)+str(m).zfill(2)+"SUM")

['PREC20040101.tif', 'PREC20040102.tif', 'PREC20040103.tif', 'PREC20040104.tif', 'PREC20040105.tif', 'PREC20040106.tif', 'PREC20040107.tif', 'PREC20040108.tif', 'PREC20040109.tif', 'PREC20040110.tif', 'PREC20040111.tif', 'PREC20040112.tif', 'PREC20040113.tif', 'PREC20040114.tif', 'PREC20040115.tif', 'PREC20040116.tif', 'PREC20040117.tif', 'PREC20040118.tif', 'PREC20040119.tif', 'PREC20040120.tif', 'PREC20040121.tif', 'PREC20040122.tif', 'PREC20040123.tif', 'PREC20040124.tif', 'PREC20040125.tif', 'PREC20040126.tif', 'PREC20040127.tif', 'PREC20040128.tif', 'PREC20040129.tif', 'PREC20040130.tif', 'PREC20040131.tif']
['PREC20040201.tif', 'PREC20040202.tif', 'PREC20040203.tif', 'PREC20040204.tif', 'PREC20040205.tif', 'PREC20040206.tif', 'PREC20040207.tif', 'PREC20040208.tif', 'PREC20040209.tif', 'PREC20040210.tif', 'PREC20040211.tif', 'PREC20040212.tif', 'PREC20040213.tif', 'PREC20040214.tif', 'PREC20040215.tif', 'PREC20040216.tif', 'PREC20040217.tif', 'PREC20040218.tif', 'PREC20040219.tif'

KeyboardInterrupt: 

In [ ]:
sr = arcpy.SpatialReference(4326) #Spatial Projection WGS84

# Set output coordinate system
outCS = arcpy.SpatialReference('NAD 1983 UTM Zone 12N')  #NAD83 Zone 12 Code is 26912

# Set local variables
inMaskData = "H:/GIS/SNODAS/SNODAS.gdb/UT_HUC_area"

# from Table 4 of http://nsidc.org/data/docs/noaa/g02158_snodas_snow_cover_model/
prodcodelong = {1025: 'Precipitation', 1034: 'Snow water equivalent', 1036: 'Snow depth', 
            1038: 'Snow pack average temperature', 1039: 'Blowing snow sublimation',
            1044: 'Snow melt', 1050: 'Snow pack sublimation'}
prodcode = {1025: 'PREC', 1034: 'SWEQ', 1036: 'SNOD', 1038: 'SPAT', 1039: 'BSSB', 1044: 'SNML', 1050: 'SPSB'}

# iterate through data files creating rasters and defining projections
for dtfile in glob.glob("H:/GIS/SNODAS/SNODASUNZIPPED/SNODASUNGZ/*.dat"):
    indata = arcpy.Raster(dtfile)
    arcpy.DefineProjection_management(indata, sr)
    typAbb = prodcode[int(os.path.basename(dtfile)[8:12])]
    
    # Determine the new output feature class path and name = productcode + YYYYMMDD and save to a geodatabase
    outrast = "H:/GIS/SNODAS/SNODAS.gdb/" + typAbb + dtfile[-20:-11] #os.path.join(outWorkspace, rast)
    
    # Print name to veryfy save
    print(typAbb + dtfile[-20:-11])
    
    # Project Raster to UTM Zone 12
    arcpy.ProjectRaster_management(outExtractByMask, outrast, outCS, 'BILINEAR', '1000',\
                                   'WGS_1984_(ITRF00)_To_NAD_1983', '#', '#')
    

In [48]:

arcpy.env.overwriteOutput = True
path="H:/GIS/SNODAS/SNODASproj.gdb/"
arcpy.env.workspace = path
for name in arcpy.ListRasters():

[u'SWEQ200309MED',
 u'SWEQ200310MED',
 u'SWEQ200311MED',
 u'SWEQ200312MED',
 u'SWEQ200401MED',
 u'SWEQ200402MED',
 u'SWEQ200403MED',
 u'SWEQ200404MED',
 u'SWEQ200405MED',
 u'SWEQ200406MED',
 u'SWEQ200407MED',
 u'SWEQ200408MED',
 u'SWEQ200409MED',
 u'SWEQ200410MED',
 u'SWEQ200411MED',
 u'SWEQ200412MED',
 u'SWEQ200501MED',
 u'SWEQ200502MED',
 u'SWEQ200503MED',
 u'SWEQ200504MED',
 u'SWEQ200505MED',
 u'SWEQ200506MED',
 u'SWEQ200507MED',
 u'SWEQ200508MED',
 u'SWEQ200509MED',
 u'SWEQ200510MED',
 u'SWEQ200511MED',
 u'SWEQ200512MED',
 u'SWEQ200601MED',
 u'SWEQ200602MED',
 u'SWEQ200603MED',
 u'SWEQ200604MED',
 u'SWEQ200605MED',
 u'SWEQ200606MED',
 u'SWEQ200607MED',
 u'SWEQ200608MED',
 u'SWEQ200609MED',
 u'SWEQ200610MED',
 u'SWEQ200611MED',
 u'SWEQ200612MED',
 u'SWEQ200701MED',
 u'SWEQ200702MED',
 u'SWEQ200703MED',
 u'SWEQ200704MED',
 u'SWEQ200705MED',
 u'SWEQ200706MED',
 u'SWEQ200707MED',
 u'SWEQ200708MED',
 u'SWEQ200709MED',
 u'SWEQ200710MED',
 u'SWEQ200711MED',
 u'SWEQ200712MED',
 u'SWEQ20080

In [52]:
monthRange = [1,12] 
yearRange = [2003,2016]

g = {}
path="H:/GIS/SNODAS/SNODASproj.gdb/"
arcpy.env.workspace = path
arcpy.env.overwriteOutput = True


for y in range(yearRange[0],yearRange[1]+1): #set years converted here
    for m in range(monthRange[0],monthRange[1]+1): #set months converted here
        my = str(y)+str(m).zfill(2)
        newdn = 'TPPT' + my
        try:
            calc = Plus('SNOW'+ my +'SUM', 'RAIN'+ my +'SUM')
            calc.save(newdn+'SUM')
            print(newdn)
        except(RuntimeError):
            pass

TPPT200309
TPPT200310
TPPT200311
TPPT200312
TPPT200401
TPPT200402
TPPT200403
TPPT200404
TPPT200405
TPPT200406
TPPT200407
TPPT200408
TPPT200409
TPPT200410
TPPT200411
TPPT200412
TPPT200501
TPPT200502
TPPT200503
TPPT200504
TPPT200505
TPPT200506
TPPT200507
TPPT200508
TPPT200509
TPPT200510
TPPT200511
TPPT200512
TPPT200601
TPPT200602
TPPT200603
TPPT200604
TPPT200605
TPPT200606
TPPT200607
TPPT200608
TPPT200609
TPPT200610
TPPT200611
TPPT200612
TPPT200701
TPPT200702
TPPT200703
TPPT200704
TPPT200705
TPPT200706
TPPT200707
TPPT200708
TPPT200709
TPPT200710
TPPT200711
TPPT200712
TPPT200801
TPPT200802
TPPT200803
TPPT200804
TPPT200805
TPPT200806
TPPT200807
TPPT200808
TPPT200809
TPPT200810
TPPT200811
TPPT200812
TPPT200901
TPPT200902
TPPT200903
TPPT200904
TPPT200905
TPPT200906
TPPT200907
TPPT200908
TPPT200909
TPPT200910
TPPT200911
TPPT200912
TPPT201001
TPPT201002
TPPT201003
TPPT201004
TPPT201005
TPPT201006
TPPT201007
TPPT201008
TPPT201009
TPPT201010
TPPT201011
TPPT201012
TPPT201101
TPPT201102
TPPT201103

In [ ]:
from arcpy.sa import *

monthRange = [1,12] 
yearRange = [2005,2005]

path = "H:/GIS/Calc.gdb/"
path1 = "H:/GIS/SNODAS/SNODASproj.gdb/"
path2 = "H:/GIS/MODIS/MODOUT.gdb/"
arcpy.env.workspace = path
arcpy.env.overwriteOutput = True
area = 'H:/GIS/NHD_UT_Proj.gdb/UT_HUC_area'
arcpy.env.mask = area

for y in range(yearRange[0],yearRange[1]+1): #set years converted here
    for m in range(monthRange[0],monthRange[1]+1): #set months converted here
        my = str(y)+str(m).zfill(2)
        newdn = 'AVLW' + my
        rain = (path1 + 'RAIN'+ my +'SUM')
        snowMelt = (path1 + 'SNML'+ my +'SUM')
        actEvap = (path2 + 'AET'+ my)
        
        avail = (Con(IsNull(rain),0, rain)) + (Con(IsNull(snowMelt),0, snowMelt)) - (Con(IsNull(actEvap),0, actEvap))
        avail = (available < 0, 0, available)
        available.save(newdn)
        print(newdn)
